## calculate vector fields to display in napari

In [34]:
def optical_flow_vector_field(path_in, path_out, starting_time = 0,  time_displacement = 1):
        
    from tifffile import imread, imwrite
    from pathlib import Path
    import numpy as np
    from skimage.registration import optical_flow_ilk

    
    path_in = Path(path_in)
    path_out = Path(path_out)
    
    #load raw image
    image = imread(path_in)
    
    #subset time points
    im_t1 = image[starting_time,:,:]
    im_t2 = image[starting_time + time_displacement,:,:]
    
    # --- Compute the optical flow
    v, u = optical_flow_ilk(im_t1, im_t2, radius=15)

    # --- Compute flow magnitude
    norm = np.sqrt(u ** 2 + v ** 2)
    
    nvec = 20  # Number of vectors to be displayed along each image dimension

    nl, nc = im_t1.shape
    step = max(nl//nvec, nc//nvec)
    
    y, x = np.mgrid[:nl:step, :nc:step]
    u_ = u[::step, ::step]
    v_ = v[::step, ::step]
    
    vectors = np.zeros((y.shape[0]*y.shape[0], 2, 2), dtype=np.float32)


    for vec1 in reversed(range(0, y.shape[0])):
        for vec2 in reversed(range(0, y.shape[0])):
            vector_id = vec1 + 20 * vec2
            vectors[vector_id, 0, 1] = x[vec2, vec1]
            vectors[vector_id, 0, 0] = y[vec2, vec1]
            vectors[vector_id, 1, 1] = u_[vec2, vec1]
            vectors[vector_id, 1, 0] = v_[vec2, vec1]
        
    #imwrite(path_out, vectors)
        
    return im_t1, norm, vectors 

In [37]:
path_in = '../../data/placozoan-movie.tif'
path_out = '../../data/placozoan_vector.tif'

im_t1, norm, vectors = optical_flow_vector_field(path_in, path_out, 50, 5)

viewer = Viewer()
viewer.add_image(im_t1)
viewer.add_image(norm, colormap = 'viridis')
layer = viewer.add_vectors(vectors)
